In [ ]:
!pip install dotenv

In [1]:
# Importing Dependencies
from bs4 import BeautifulSoup
from splinter import Browser
import pandas as pd
from datetime import datetime, timedelta
from pprint import pprint
import time
import os
import numpy as np
from dotenv import load_dotenv
load_dotenv()
import os
import numpy as np

In [2]:
os.getenv('password')

'4rFR!gPHpY4tU8y'

In [28]:
#user = os.getenv('USER')
user = 'asoemardy@gmail.com'
#password = os.getenv('password')
password = 'Soem@ce91719'

In [29]:
#Create the browser
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)

try:

    #Visit the website in 'url'
    url = 'https://www.appannie.com/dashboard/home'
    browser.visit(url)

    #Enter login info
    time.sleep(1)
    browser.find_by_name(name = 'username').type(user)
    browser.find_by_css('input')[1].type(password)
    browser.find_by_tag('button').click()

    #########
    def checker(func,go=None):
        #func is a 'is_present' boolean
        for i in range(3):
            if func:
                go
                break
            else:
                browser.reload()
                time.sleep(2)
    ##########

    #Check for presence of 'Top Chart' tab, click tab
    checker((browser.is_element_present_by_text('Top Charts',wait_time=2)),browser.find_by_text('Top Charts').click())

    #Create date range
    today = datetime.now()
    dates = [(today - timedelta(days=i)).strftime("%Y-%m-%d") for i in range(90)]



    #https://www.appannie.com/apps/ios/top-chart?country=US&category=36&device=iphone&date=2020-12-01&feed=All&page_number=0&page_size=500&rank_sorting_type=rank
    #list of dictionaries, each one a different day
    list_of_builds = []

    for moment in dates:
        #get top chart info
        url = f'https://www.appannie.com/apps/ios/top-chart?country=US&category=36&device=iphone&date={moment}&feed=All&page_number=0&page_size=500&rank_sorting_type=rank'
        browser.visit(url)
        time.sleep(5)
        html = browser.html
        soup = BeautifulSoup(html,'html.parser')

        checker(browser.is_element_present_by_css('table'))

        all_trs = soup.table.tbody.find_all('tr')
        build_df = {'Rank':[],'Date': moment,'Free':[],'Paid':[],'Grossing':[]}
        for _ in all_trs:
            current_rank_apps = [elem.text for elem in _.select('a span')]
            
            toss, app_id1, app_id2, app_id3 = [i.get('data-appid') for i in _.find_all('td')]
            
            rank_num = _.span.text
            build_df['Rank'].append((rank_num))
            build_df['Free'].append([current_rank_apps[0],current_rank_apps[1],app_id1])
            build_df['Paid'].append([current_rank_apps[2],current_rank_apps[3],app_id2])
            build_df['Grossing'].append([current_rank_apps[4],current_rank_apps[5],app_id3])

        list_of_builds.append(build_df)

    #concatenate all dataframes
    builds_to_concat = [pd.DataFrame(_).set_index('Date') for _ in list_of_builds]
    sep3_dec1_t500_ios = pd.concat(builds_to_concat)
except Exception as e:
    browser.quit()
    print(e)
finally:
    browser.quit()

In [14]:
sep3_dec1_t500.to_csv('top_500_90day.csv')

In [30]:
sep3_dec1_t500

,Rank,Free,Paid,Grossing
Date,,,,
2020-12-02,1,"[ZOOM Cloud Meetings, Zoom Video Communication...","[Minecraft, Mojang, 479516143]","[YouTube: Watch, Listen, Stream, Google, 54400..."
2020-12-02,2,"[Roof Rails, Voodoo, 1524760095]","[Bloons TD 6, Ninja Kiwi, 1118115766]","[Tinder - Dating & Make Friends, Match Group, ..."
2020-12-02,3,"[YouTube: Watch, Listen, Stream, Google, 54400...","[Monopoly, Marmalade, 1477966166]","[Disney+, Disney, 1446075923]"
2020-12-02,4,"[TikTok, TikTok Pte Ltd, 835599320]","[Plague Inc., Ndemic Creations, 525818839]","[Pokémon GO, Niantic, 1094591345]"
2020-12-02,5,"[Spotify: Music and podcasts, Spotify, 324684580]","[Procreate Pocket, Savage Interactive, 916366645]","[Clash of Clans, Supercell, 529479190]"
...,...,...,...,...
2020-09-04,496,"[Smule: Social Karaoke Singing, Smule, 509993510]","[Bridge Constructor Portal, HeadUpGames, 13113...","[BothLive, Zhiji Network, 1445657647]"
2020-09-04,497,"[SmartThings, Samsung Electronics, 1222822904]","[Spy hidden camera Detector, Jakobi Teknik, 92...","[Satellite VPN: Secure Browsing, Syazan OU, 15..."
2020-09-04,498,"[Fall Guys : Knockout Ultimate, Jpseph Rader, ...","[The Impossible Game, FlukeDude, 367242040]","[Bike Race: Free Style Games, Wildlife Studios..."


In [31]:
df_raw_new = sep3_dec1_t500.reset_index()
df_raw_new

,Date,Rank,Free,Paid,Grossing
0,2020-12-02,1,"[ZOOM Cloud Meetings, Zoom Video Communication...","[Minecraft, Mojang, 479516143]","[YouTube: Watch, Listen, Stream, Google, 54400..."
1,2020-12-02,2,"[Roof Rails, Voodoo, 1524760095]","[Bloons TD 6, Ninja Kiwi, 1118115766]","[Tinder - Dating & Make Friends, Match Group, ..."
2,2020-12-02,3,"[YouTube: Watch, Listen, Stream, Google, 54400...","[Monopoly, Marmalade, 1477966166]","[Disney+, Disney, 1446075923]"
3,2020-12-02,4,"[TikTok, TikTok Pte Ltd, 835599320]","[Plague Inc., Ndemic Creations, 525818839]","[Pokémon GO, Niantic, 1094591345]"
4,2020-12-02,5,"[Spotify: Music and podcasts, Spotify, 324684580]","[Procreate Pocket, Savage Interactive, 916366645]","[Clash of Clans, Supercell, 529479190]"
...,...,...,...,...,...
44995,2020-09-04,496,"[Smule: Social Karaoke Singing, Smule, 509993510]","[Bridge Constructor Portal, HeadUpGames, 13113...","[BothLive, Zhiji Network, 1445657647]"
44996,2020-09-04,497,"[SmartThings, Samsung Electronics, 1222822904]","[Spy hidden camera Detector, Jakobi Teknik, 92...","[Satellite VPN: Secure Browsing, Syazan OU, 15..."
44997,2020-09-04,498,"[Fall Guys : Knockout Ultimate, Jpseph Rader, ...","[The Impossible Game, FlukeDude, 367242040]","[Bike Race: Free Style Games, Wildlife Studios..."
44998,2020-09-04,499,"[Afterpay - Buy now. Pay later., Afterpay, 140...","[Site Audit Pro, Veam Studios, 430234732]","[Microsoft OneDrive, Microsoft, 477537958]"


In [36]:
def check_publisher(element, element_list):
    if element not in element_list:
        element_list.append(element)
        
def check_app(element, cat, element_list, cat_list, lookup_table, pub_id_list, app_id_list):
    if element[0] not in element_list:
        element_list.append(element[0])
        cat_list.append(cat)
        app_id_list.append(element[2])
        pub_id_list.append(int(lookup_table[lookup_table['Publisher_Name'] == element[1]]['Publisher_ID']))
        
    

In [26]:
temp = df_raw_new['Free'][0].split("'")
if temp[3] not in df_publisher:
    print(df_publisher[df_publisher['Publisher_Name'] == temp[1]]['Publisher_ID'])

0    1
Name: Publisher_ID, dtype: int64


In [37]:
publisher_list = []

for index, row in df_raw_new.iterrows():
    check_publisher(row['Free'][1], publisher_list)
    check_publisher(row['Grossing'][1], publisher_list)
    check_publisher(row['Paid'][1], publisher_list)
   



In [38]:
publisher_id = np.arange(1, len(publisher_list)+1)

In [39]:
publisher_list

['Zoom Video Communications',
 'Google',
 'Mojang',
 'Voodoo',
 'Match Group',
 'Ninja Kiwi',
 'Disney',
 'Marmalade',
 'TikTok Pte Ltd',
 'Niantic',
 'Ndemic Creations',
 'Spotify',
 'Supercell',
 'Savage Interactive',
 'Instagram',
 'miHoYo',
 'Warner Bros',
 'InnerSloth',
 'Roblox',
 'RobTop',
 'Facebook',
 'Pandora',
 'HotSchedules',
 'Hulu',
 'Rockstar Games',
 'King',
 'Lightricks',
 'Snap',
 'Bumble',
 'Jacquielawson',
 'Amazon',
 'Netflix',
 'ADVA Soft',
 'Shopify',
 'Playrix',
 'True Axis',
 'Square Inc',
 '2K Games',
 'Warner Media',
 'Domus Technica',
 'WhatsApp',
 'Twitch Interactive',
 'Seekrtech',
 'SayGames',
 'Moon Active',
 '505 Games',
 'BANDAI NAMCO Entertainment',
 'Dual Decade',
 'PayPal',
 'BIGO',
 'Tantsissa',
 'Walmart',
 'Peak',
 'Snoozle Limited',
 'Clickteam',
 'Activision Publishing',
 'Tencent',
 'Matt Berg',
 'Venmo',
 'Playtika',
 'Suture Sound Inc.',
 'Twitter',
 'Garena Online',
 'Chucklefish',
 'Works Inc',
 'Bubblegum Games',
 'Apple',
 'Microsoft',
 

In [40]:
df_publisher = pd.DataFrame(zip(publisher_id, publisher_list), columns=['Publisher_ID','Publisher_Name'])
df_publisher

,Publisher_ID,Publisher_Name
0,1,Zoom Video Communications
1,2,Google
2,3,Mojang
3,4,Voodoo
4,5,Match Group
...,...,...
2491,2492,Queenie Law
2492,2493,Justyna Zablocka
2493,2494,bopdrop
2494,2495,Amrita Studio


In [41]:
app_list = []
cat_list = []
pub_id = []
app_id = []

for index, row in df_raw_new.iterrows():
    # If the app is not in the list yet, insert it into the list and insert the category on the cat_list: 1(Free), 2(Grossing), 3(Paid)
    check_app(row['Free'], 1, app_list, cat_list, df_publisher, pub_id, app_id)
    check_app(row['Grossing'], 2, app_list, cat_list, df_publisher, pub_id, app_id)
    check_app(row['Paid'], 3, app_list, cat_list, df_publisher, pub_id, app_id)

In [42]:
df_app = pd.DataFrame(zip(app_id, app_list, cat_list, pub_id), columns=['App_ID','App_Name','Category','Publisher_ID'])
df_app

,App_ID,App_Name,Category,Publisher_ID
0,546505307,ZOOM Cloud Meetings,1,1
1,544007664,"YouTube: Watch, Listen, Stream",2,2
2,479516143,Minecraft,3,3
3,1524760095,Roof Rails,1,4
4,547702041,Tinder - Dating & Make Friends,2,5
...,...,...,...,...
3691,1515694739,GetMagic - Flash Likes Effects,1,2492
3692,1509682673,NG Flight Simulator,3,2493
3693,1493303435,bopdrop - music discovery,1,2494
3694,1494881259,Campus: Date Sim,2,2495


In [43]:
df_free = pd.DataFrame(df_raw_new['Free'].to_list(), columns = ['App_Name', 'Publisher', 'App_ID'])
df_free

,App_Name,Publisher,App_ID
0,ZOOM Cloud Meetings,Zoom Video Communications,546505307
1,Roof Rails,Voodoo,1524760095
2,"YouTube: Watch, Listen, Stream",Google,544007664
3,TikTok,TikTok Pte Ltd,835599320
4,Spotify: Music and podcasts,Spotify,324684580
...,...,...,...
44995,Smule: Social Karaoke Singing,Smule,509993510
44996,SmartThings,Samsung Electronics,1222822904
44997,Fall Guys : Knockout Ultimate,Jpseph Rader,1528248331
44998,Afterpay - Buy now. Pay later.,Afterpay,1401019110


In [45]:
df_free_cleaned = df_raw_new[['Date','Rank']].merge(df_free[['App_ID']], left_index = True, right_index = True)
df_free_cleaned

,Date,Rank,App_ID
0,2020-12-02,1,546505307
1,2020-12-02,2,1524760095
2,2020-12-02,3,544007664
3,2020-12-02,4,835599320
4,2020-12-02,5,324684580
...,...,...,...
44995,2020-09-04,496,509993510
44996,2020-09-04,497,1222822904
44997,2020-09-04,498,1528248331
44998,2020-09-04,499,1401019110


In [57]:
df_paid = pd.DataFrame(df_raw_new['Paid'].to_list(), columns = ['App_Name', 'Publisher', 'App_ID'])
df_paid

,App_Name,Publisher,App_ID
0,Minecraft,Mojang,479516143
1,Bloons TD 6,Ninja Kiwi,1118115766
2,Monopoly,Marmalade,1477966166
3,Plague Inc.,Ndemic Creations,525818839
4,Procreate Pocket,Savage Interactive,916366645
...,...,...,...
44995,Bridge Constructor Portal,HeadUpGames,1311353234
44996,Spy hidden camera Detector,Jakobi Teknik,925967783
44997,The Impossible Game,FlukeDude,367242040
44998,Site Audit Pro,Veam Studios,430234732


In [58]:
df_paid_cleaned = df_raw_new[['Date','Rank']].merge(df_paid[['App_ID']], left_index = True, right_index = True)
df_paid_cleaned

,Date,Rank,App_ID
0,2020-12-02,1,479516143
1,2020-12-02,2,1118115766
2,2020-12-02,3,1477966166
3,2020-12-02,4,525818839
4,2020-12-02,5,916366645
...,...,...,...
44995,2020-09-04,496,1311353234
44996,2020-09-04,497,925967783
44997,2020-09-04,498,367242040
44998,2020-09-04,499,430234732


In [59]:
df_grossing = pd.DataFrame(df_raw_new['Grossing'].to_list(), columns = ['App_Name', 'Publisher', 'App_ID'])
df_grossing

,App_Name,Publisher,App_ID
0,"YouTube: Watch, Listen, Stream",Google,544007664
1,Tinder - Dating & Make Friends,Match Group,547702041
2,Disney+,Disney,1446075923
3,Pokémon GO,Niantic,1094591345
4,Clash of Clans,Supercell,529479190
...,...,...,...
44995,BothLive,Zhiji Network,1445657647
44996,Satellite VPN: Secure Browsing,Syazan OU,1515543740
44997,Bike Race: Free Style Games,Wildlife Studios,510461758
44998,Microsoft OneDrive,Microsoft,477537958


In [60]:
df_grossing_cleaned = df_raw_new[['Date','Rank']].merge(df_grossing[['App_ID']], left_index = True, right_index = True)
df_grossing_cleaned

,Date,Rank,App_ID
0,2020-12-02,1,544007664
1,2020-12-02,2,547702041
2,2020-12-02,3,1446075923
3,2020-12-02,4,1094591345
4,2020-12-02,5,529479190
...,...,...,...
44995,2020-09-04,496,1445657647
44996,2020-09-04,497,1515543740
44997,2020-09-04,498,510461758
44998,2020-09-04,499,477537958


In [5]:
#Create the browser
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)

try:

    #Visit the website in 'url'
    url = 'https://www.appannie.com/dashboard/home'
    browser.visit(url)

    #Enter login info
    time.sleep(1)
    browser.find_by_name(name = 'username').type(os.getenv('user'))
    browser.find_by_css('input')[1].type(os.getenv('password'))
    browser.find_by_tag('button').click()

    #########
    def checker(func,go=None):
        #func is a 'is_present' boolean
        for i in range(3):
            if func:
                go
                break
            else:
                browser.reload()
                time.sleep(2)
    ##########

    #Check for presence of 'Top Chart' tab, click tab
    checker((browser.is_element_present_by_text('Top Charts',wait_time=2)),browser.find_by_text('Top Charts').click())

    #Create date range
    today = datetime.now()
    dates = [(today - timedelta(days=i)).strftime("%Y-%m-%d") for i in range(90)]



    #https://www.appannie.com/apps/ios/top-chart?country=US&category=36&device=iphone&date=2020-12-01&feed=All&page_number=0&page_size=500&rank_sorting_type=rank
    #list of dictionaries, each one a different day
    list_of_builds = []

    for moment in dates:
        #get top chart info
        url = f'https://www.appannie.com/apps/google-play/top-chart/?country=US&category=1&device=&date={moment}&feed=All&rank_sorting_type=rank&page_number=0&page_size=100'
        browser.visit(url)
        time.sleep(5)
        html = browser.html
        soup = BeautifulSoup(html,'html.parser')

        checker(browser.is_element_present_by_css('table'))

        all_trs = soup.table.tbody.find_all('tr')
        build_df = {'Rank':[],'Date': moment,'Free':[],'Paid':[],'Grossing':[]}
        for _ in all_trs:
            current_rank_apps = [elem.text for elem in _.select('a span')]
            
            toss, app_id1, app_id2, app_id3, app_id4, app_id5 = [i.get('data-appid') for i in _.find_all('td')]
            
            rank_num = _.span.text
            build_df['Rank'].append((rank_num))
            build_df['Free'].append([current_rank_apps[0],current_rank_apps[1],app_id1])
            build_df['Paid'].append([current_rank_apps[2],current_rank_apps[3],app_id2])
            build_df['Grossing'].append([current_rank_apps[4],current_rank_apps[5],app_id3])

        list_of_builds.append(build_df)

    #concatenate all dataframes
    builds_to_concat = [pd.DataFrame(_).set_index('Date') for _ in list_of_builds]
    sep3_dec1_t500_google_play = pd.concat(builds_to_concat)
except Exception as e:
    browser.quit()
    print(e)
finally:
    browser.quit()

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('')